In [20]:
import numpy as numpy

DEFINITION OF THE CONSTANTS OF MIRAGE 3

<div style="text-align: center;">
    <img src="images/dm3.jpg" alt="MIRAGE III" width="600">
</div>


In [21]:
l_ref = 5.24             # reference lenght in meters
l_t   = (3/2) * l_ref    # total lenght in meters
m     = 8400             # mass in kg
c     = 0.52             # center of gravity position as a pourcentage of total lenght
S     = 34               # wings surface in m^2
r_g   = 2.65             # radius of gyration in m

OUR CHOOSEN OPERATING POINT WILL BE 16 : 
Mach 1.71
Alt  511 ft

<div style="text-align: center;">
    <img src="images/aircraft-aerodynamic-points.png" alt="Aerodynamical points" width="600">
</div>

We want to determine the equilibrium conditions around the operation point. We will use the model with a state space representation (A,B,C,D). We consider the following vectors : 

### State Vector


\begin{equation*}
X = \begin{pmatrix}
V ~
\gamma ~
\alpha ~
q ~
\theta~ 
z~
\end{pmatrix}
\end{equation*}


### Command Vector


\begin{equation*}
U = \begin{pmatrix}
\delta_m
\end{pmatrix}
\end{equation*}
According to the graphs : 

\begin{equation*}
C_{x0} = 0.03 
\end{equation*}

\begin{equation}
C_{z\alpha} = 2.23
\end{equation}

\begin{equation}
C_{\delta_m} = 0.38 ~~ [rad^{-1}]
\end{equation}

\begin{equation}
\delta_{m0} = -0.007 ~~ [rad]
\end{equation}

\begin{equation}
\alpha_0 = 0.01 ~~ [rad]
\end{equation}

\begin{equation}
f = 0.608 
\end{equation}

\begin{equation}
f_{\delta} = 0.9
\end{equation}

\begin{equation}
k = 0.48
\end{equation}

\begin{equation}
C_{mq} = -0.27 ~~ [s/rad]
\end{equation}

In [26]:
l_ref = 5.24             # reference lenght in meters
l_t   = (3/2) * l_ref    # total lenght in meters
m     = 8400             # mass in kg
c     = 0.52             # center of gravity position as a pourcentage of total lenght
S     = 34               # wings surface in m^2
r_g   = 2.65             # radius of gyration in m


g0       = 9.81
rho0     = 1.225
R_air    = 287.0
gamma_air= 1.4
T0       = 288.15
h_eq     = 511 * 0.3048
Mach_eq  = 1.71

In [27]:
# --- coefficients fournis ---
C_x0         = 0.03
k            = 0.48
C_z_alpha    = -2.23    # dCz/dalpha [per rad] (utilise la valeur que tu as écrite)
C_z_delta_m  = 0.38     # dCz/ddelta_m [per rad]
delta_m0     = -0.007   # δm initial (rad)
f            = 0.608
c            = 0.52
f_delta      = 0.9
alpha0       = 0.01     # alpha0 initial (rad)

# calculs géométriques / bras de levier
x_g    = c * l_t            # position du centre de gravité (m)
x_f    = f * l_t            # position d'action de la force (m)
X      = x_f - x_g          # bras de levier utilisé dans la formule
x_fdelta = f_delta * l_t
Y      = x_fdelta - x_g

# itération / tolérances
eps = 1e-6
max_iter = 200

In [28]:
import math
import pandas as pd
f=0.61 # Corrected: removed the trailing comma from the global definition
def isa_density(h, T0=288.15, p0=101325.0, L=0.0065, R=287.0, g=9.81):
    """Simple ISA troposphere density (valid up to 11 km)."""
    T = T0 - L * h
    p = p0 * (T / T0) ** (g / (R * L))
    rho = p / (R * T)
    return rho, T

def compute_equilibrium():

    # atmosphere at altitude
    rho, T = isa_density(h_eq, T0=T0, R=R_air, g=g0)
    a = math.sqrt(gamma_air * R_air * T)   # speed of sound
    V = Mach_eq * a
    Q = 0.5 * rho * V**2

    # initialization
    alpha = 0.0  # alpha_eq0 [rad]
    F_px = 0.0   # thrust axial force initial

    history = []

    for i in range(max_iter):
        # 1) Cz_eq from weight balance (see flowchart)
        C_z_eq = (m * g0 - F_px * math.sin(alpha)) / (Q * S)

        # 2) Cx_eq (nonlinear drag/axial model in flowchart)
        C_x_eq = C_x0 + k * C_z_eq**2

        # 3) Cx_delta_m (depends on C_z_eq per flowchart right box)
        C_x_delta_m = 2.0 * k * C_z_eq * C_z_delta_m

        # 4) delta_m equilibrium (from flowchart)
        numerator = C_x_eq * math.sin(alpha) + C_z_eq * math.cos(alpha)
        denominator = C_x_delta_m * math.sin(alpha) + C_z_delta_m * math.cos(alpha)
        if abs(denominator) < 1e-12:
            raise ZeroDivisionError("Denominator near zero when computing delta_m — check coefficients/geometry.")

        delta_m_eq = delta_m0 - (numerator / denominator) * (X / (Y - X))

        # 5) alpha_{i+1}
        alpha_next = alpha0 + (C_z_eq / C_z_alpha) - (C_z_delta_m / C_z_alpha) * delta_m_eq

        # 6) thrust axial at next step (flowchart)
        # avoid cos ~ 0 -> check
        if abs(math.cos(alpha_next)) < 1e-6:
            raise ValueError("cos(alpha_next) too small, alpha close to 90 deg — invalid regime for this model.")
        F_px_next = Q * S * C_x_eq / math.cos(alpha_next)

        # store history
        history.append({
            'iter': i,
            'alpha_rad': alpha,
            'alpha_deg': math.degrees(alpha),
            'Cz_eq': C_z_eq,
            'Cx_eq': C_x_eq,
            'delta_m_eq_rad': delta_m_eq,
            'delta_m_eq_deg': math.degrees(delta_m_eq),
            'F_px': F_px,
        })

        # convergence check
        if abs(alpha_next - alpha) < eps:
            alpha = alpha_next
            F_px = F_px_next
            break

        # update for next iteration
        alpha = alpha_next
        F_px = F_px_next
    else:
        # max iterations reached
        i = max_iter - 1

    # final values
    result = {
        'alpha_eq_rad': alpha,
        'alpha_eq_deg': math.degrees(alpha),
        'delta_m_eq_rad': delta_m_eq,
        'delta_m_eq_deg': math.degrees(delta_m_eq),
        'Cz_eq': C_z_eq,
        'Cx_eq': C_x_eq,
        'C_x_delta_m': C_x_delta_m,
        'F_px': F_px,
        'Q': Q,
        'V': V,
        'rho': rho,
        'iterations': i+1,
        'X': X,
        'Y': Y,
        'converged': abs(alpha_next - alpha) < eps
    }

    # present a small table of the last few iterations
    df_hist = pd.DataFrame(history)
    return result, df_hist

# --- Example run with the user-provided vehicle parameters and placeholder derivatives ---
res, hist = compute_equilibrium()

# show final result and last iterations
print("Equilibrium result (example coefficients):")
for k,v in res.items():
    if isinstance(v, float):
        print(f"  {k:15s} = {v:.6g}")
    else:
        print(f"  {k:15s} = {v}")

display(hist.tail(8))

Equilibrium result (example coefficients):
  alpha_eq_rad    = 0.00188564
  alpha_eq_deg    = 0.108039
  delta_m_eq_rad  = -0.0164408
  delta_m_eq_deg  = -0.941987
  Cz_eq           = 0.0118475
  Cx_eq           = 0.0300674
  C_x_delta_m     = 0.00432198
  F_px            = 208134
  Q               = 203596
  V               = 580.826
  rho             = 1.207
  iterations      = 3
  X               = 0.69168
  Y               = 2.9868
  converged       = True


,iter,alpha_rad,alpha_deg,Cz_eq,Cx_eq,delta_m_eq_rad,delta_m_eq_deg,F_px
0,0,0.000000,0.000000,0.011904,0.030068,-0.016441,-0.941998,0.000000
1,1,0.001860,0.106581,0.011848,0.030067,-0.016441,-0.941987,208138.920625
2,2,0.001885,0.108020,0.011848,0.030067,-0.016441,-0.941987,208134.516123
